# PROYECTO COVID-19

Imports necesarios para el correcto funcionamiento del notebook

In [1]:
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
import os
import shutil
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
%matplotlib inline
import time
from skimage.feature import hog

# Primera aproximación del problema

Función que permite la lectura de imágenes a partir del directorio en el que estan alamacenadas:

In [2]:
def lectura(subdirectorio,fichero,path="Dataset/"):
    if(os.path.exists(path+subdirectorio)):
        listaImagenes = os.listdir(path+subdirectorio)
        datos = []

        for imagen in listaImagenes:
            datos.append(cv2.imread(path+subdirectorio+imagen,0))
        if(os.path.isfile(path+fichero)):
            with open(path+fichero) as fp: 
                salidas = fp.read().splitlines()
            return np.array(datos), np.array(salidas)
        else:
            print("¡Error! El fichero de salidas no existe")
    else:
        print("¡Error! El directorio no existe")

Definimos la funcion para leer imagenes y aumentar el tamaño de nuestro conjunto de datos, utilizando:
- Ruido Gaussiano
- Volteando verticalmente

In [3]:
def tratamiento(pathImagenes="Data/", dx=256, dy=256, noiseX=5, noiseY=10):
    if(os.path.exists(pathImagenes)):
        resultado = []
        listaImagenes = os.listdir(pathImagenes)

        #Realizar un data augmentation para aumentar los datos del dataset:
        gaussian_noise = iaa.AdditiveGaussianNoise(noiseX, noiseY)
        flip_vr=iaa.Fliplr(p=1.0)

        for imagen in listaImagenes:
            Xray = cv2.resize(cv2.imread(pathImagenes+imagen,0),(dx, dy))
            resultado.append(Xray)
            resultado.append(gaussian_noise.augment_image(Xray))
            resultado.append(flip_vr.augment_image(Xray))

        return resultado
    else:
        print("¡Error! El path especificado no ha sido encontrado")
        return None

Definimos una funcion para hacer el particionamiento de los datos en train, test y val

In [4]:
def particionamiento(listaDatos, train_percent=.6, validate_percent=.2, seed=0):
    np.random.seed(seed)
    listaTrain=[]
    listaTest=[]
    listaVal=[]
    y_train = []
    y_test = []
    y_val = []
    for i,lista in enumerate(listaDatos):
        lista = np.array(lista)
        perm = np.random.permutation(len(lista))
        m = len(lista)
        train_end = int(train_percent * m)
        val_end = int(validate_percent * m) + train_end
        train = lista[perm[:train_end]]
        val = lista[perm[train_end:val_end]]
        test = lista[perm[val_end:]]
        listaTrain.extend(train)
        listaTest.extend(test)
        listaVal.extend(val)
        y_train.extend(list([i]*len(train)))
        y_test.extend(list([i]*len(test)))
        y_val.extend(list([i]*len(val)))
    
    return listaTrain, listaTest, listaVal, y_train, y_test, y_val

Esta función permite almacenar las imágenes leidas y tratasdas durante la fase de train:

In [5]:
def almacenaImagenes(train,test,val,y_train,y_test,y_val,path="Dataset/"):
    if(os.path.exists(path)):
        try:
            shutil.rmtree(path,ignore_errors=True)
            
        except OSError as e:
            print("¡Error! No se ha podido eliminar el directorio")
    
    os.mkdir(path, 0o7777)
    os.mkdir(path+"train/",0o7777)
    os.mkdir(path+"val/",0o7777)
    os.mkdir(path+"test/",0o7777)
    
    f = open(path+"train.txt", "w")
    for i,imagen in enumerate(train):
        cv2.imwrite(path+"train/Train-"+str(i)+".png",imagen)
        f.write(str(y_train[i])+"\n")
    f.close()
    
    f = open(path+"test.txt", "w")
    for i,imagen in enumerate(test):
        cv2.imwrite(path+"test/Test-"+str(i)+".png",imagen)
        f.write(str(y_test[i])+"\n")
    f.close()
    
    f = open(path+"val.txt", "w")
    for i,imagen in enumerate(val):
        cv2.imwrite(path+"val/Val-"+str(i)+".png",imagen)
        f.write(str(y_val[i])+"\n")
    f.close()

In [6]:
def clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[0], norm=False):

    if(norm == True):
        Xtrain = Xtrain/255
        Xtest = Xtest/255
        Xval = Xval/255

    accTrain=[]
    accTest=[]
    accVal=[]
    start_time = time.time()

    for i,k in enumerate(score):
        #Seleccionar aquellos pixeles que tengan un Score mayor o igual al establecido
        pixelesSelec = df[df.Score >= k]['Pixeles'].values.tolist()

        #Entrenar el modelo con los ejemplos con el nuevo numero de caracteristicas:
        #Generar el nuevo conjunto de Train:
        X_train_tratada = Xtrain[:,pixelesSelec]

        #Entrenar el modelo:
        clasificador.fit(X_train_tratada,y_train)

        #Obtener la predcción en train y el accuracy:
        predictTrain = clasificador.predict(X_train_tratada)
        accTrain.append(metrics.accuracy_score(predictTrain,y_train)*100)

        #Generar el nuevo conjunto de Val:
        X_val_tratada = Xval[:,pixelesSelec]

        #Obtener la predcción en val y el accuracy:
        predictVal = clasificador.predict(X_val_tratada)
        accVal.append(metrics.accuracy_score(predictVal,y_val)*100)

        #Generar el nuevo conjunto de Test:
        X_test_tratada = Xtest[:,pixelesSelec]

        #Obtener la predcción en test y el accuracy:
        predictTest = clasificador.predict(X_test_tratada)
        accTest.append(metrics.accuracy_score(predictTest,y_test)*100)

        print('El rendimiento en entrenamiento con {} variables para un score de {} es de {}%'.format(len(pixelesSelec),k,accTrain[i]))
        print('El rendimiento en validacion con {} variables para un score de {}  es de {}%'.format(len(pixelesSelec),k,accVal[i]))
        print('El rendimiento en test con {} variables para un score de {}  es de {}%'.format(len(pixelesSelec),k,accTest[i]))

    tiempo_ejecucion = time.time() - start_time
    print("--- %s seconds ---" % (tiempo_ejecucion))

Esta función genera la matriz de confusión para un conjunto de datos de entrada y muestra los valores de precisión, recall y f-score

In [7]:
def genera_confusion(X,y):
    if(X == None or y == None):
        print("¡Error en los parámetros introducidos!")
        return
    
    print("********************************* Plot de matriz de confusión *********************************")
    class_names = ['COVID-19','Normal','Viral']
    titles_options = [("Matriz de confusion sin normalizar", None),
                      ("Matriz de confusion normalizada", 'true')]
    for title, normalize in titles_options:
        disp = metrics.plot_confusion_matrix(regresion, X_test_tratada, y_test,
                                     display_labels=class_names,
                                     cmap=plt.cm.Blues,
                                     normalize=normalize)
        disp.ax_.set_title(title)
        print(title)
        print(disp.confusion_matrix)
    plt.show()
    print("********************************* Valores de precisión, recall y f-score *********************************")
    print(metrics.classification_report(y_test, regresion.predict(X_test_tratada)))

Función que permite copiar ficheros entre directorios

In [8]:
def copiaImagenes(src='ImagenesCovidExtra/', dst='Data/COVID-19/'):
    if(not os.path.exists(src)):
        print("¡Error! El directorio de origen no existe")
        return
    elif(not os.path.exists(src)):
        print("¡Error! El directorio de destino no existe")
        return
    print("Copiando ficheros de "+src+" a "+dst)
    src_files = os.listdir(src)
    for file_name in src_files:
        full_file_name = os.path.join(src, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, dst)

Celda principal para la ejecución de las pruebas de la primera aproximación

In [10]:
#Constantes del problema:
pathCovid = "Data/COVID-19/"
pathNormal = "Data/NORMAL/"
pathViral = "Data/Viral Pneumonia/" 
pathImagenesExtra = "ImagenesCovidExtra/"
pathImagenes= "Dataset/"
fichero_train = "train.txt"
fichero_val = "val.txt"
fichero_test = "test.txt"
dx=256 #Tamaño de las imágenes en eje x
dy=256 #Tamaño de las imágenes en eje y
###############################################################################################################################

#Pretratamiento de imagenes y almacenamiento de las mismas:
#Realizamos el tratamiento de las imágenes:
listaCOVID = tratamiento(pathCovid,dx,dy)
listaNORMAL = tratamiento(pathNormal,dx,dy)
listaVIRAL = tratamiento(pathViral,dx,dy)

#Generamos el particionamiento
listaDatos = [listaCOVID, listaNORMAL, listaVIRAL] 
train, test, val, y_train, y_test, y_val = particionamiento(listaDatos)

#Almacenamos las imágenes en disco
almacenaImagenes(train,test,val,y_train,y_test,y_val,pathImagenes)
###############################################################################################################################

#Lectura de las imágenes desde disco:
train, y_train = lectura("train/",fichero_train,pathImagenes)
val, y_val = lectura("val/",fichero_val,pathImagenes)
test, y_test = lectura("test/",fichero_test,pathImagenes)

###############################################################################################################################

#Dejamos las imágenes en 2D para la selección de características:
Xtrain = np.reshape(train, (train.shape[0], dx*dy))
Xtest = np.reshape(test, (test.shape[0], dx*dy))
Xval = np.reshape(val, (val.shape[0], dx*dy))

#Comprobación de las características y selección:
Kbest = SelectKBest(f_classif) #Generamos el objeto SelectKBest
Kbest.fit(Xtrain, y_train) #Entrenar el modelo
numPixel = list(range(dx*dy)) #Generar el listado de número de píxeles

#Generar un dataframe donde la primera columna sean el número de pixeles o características y la segundo el score:
df = pd.DataFrame({'Pixeles': numPixel,'Score': Kbest.scores_.tolist()})
#Generar un plot de barras para mostrar la información:
plot = df.Score.plot(kind = 'hist', bins=40, title='Selección de Kbest')
plot.set_xlabel("Score")
plot.set_ylabel("Frecuencia")

###############################################################################################################################

##Pruebas##

#Probamos a clasificar con todas las características y con lo parámetros por defecto del clasificador:
print("********************************* Prueba clasificador con todas las características ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[0], norm=False)

#Probamos a clasificar normalizando los datos de entrada:
print("********************************* Prueba clasificador con todas las características normalizando ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[0], norm=True)

#Probamos con diferentes valores de score:
print("********************************* Prueba clasificador con varios valores de score ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[20,30,45,60], norm=False)

#Probamos con diferentes valores de score y normalizando los datos:
print("********************************* Prueba clasificador con varios valores de score normalizando ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[20,30,45,60], norm=True)

#Probamos para un valor de score de 60 y generamos las matrices de confusión:
print("********************************* Prueba clasificador con score 60 y normalizando ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[60], norm=True)
genera_confusion(Xtrain,y_train)
genera_confusion(Xval,y_val)
genera_confusion(Xtest,y_test)

#Hacemos un gridSearch para encontrar los mejores valores de los parámetros para la regresión logística:
print("********************************* GridSearch para score 60 y diferentes algoritmos de optimización y normalización ********************************* ")
pixelesSelec = df[df.Score >= 60]['Pixeles'].values.tolist()
X_train_tratada = Xtrain[:,pixelesSelec]
params={'solver':['newton-cg','sag', 'saga', 'lbfgs'], 'penalty':['l1', 'l2', 'elasticnet'], 'C':[1,10,100,1000]}
cv=5
clf = GridSearchCV(LogisticRegression(), params, cv=cv, scoring='accuracy', n_jobs=-1)
clf.fit(X_train_tratada,y_train)
print("EL mejor resultado obtenido ha sido: "+clf.best_score_+", los mejores parámetros obtenidos son: "+clf.best_params_)
clasificador = LogisticRegression(solver='saga',penalty="l1",C=1)
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[60], norm=True)

#Hacemos otros gridSearch de nuevo pero para liblinear ya que hemos comprobado que es la mejor función de optimización para este problema:
print("********************************* GridSearch para score 60 y diferentes algoritmos de normalización  ********************************* ")
params={'solver':['liblinear'], 'penalty':['l1', 'l2', 'elasticnet'], 'C':[10,100,1000]}
clf = GridSearchCV(LogisticRegression(), params, cv=cv, scoring='accuracy', n_jobs=-1)
clf.fit(X_train_tratada,y_train)
print("EL mejor resultado obtenido ha sido: "+clf.best_score_+", los mejores parámetros obtenidos son: "+clf.best_params_)
clasificador = LogisticRegression(solver='liblinear',penalty="l1",C=100)
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[60], norm=True)

#Añadimos nuevos ejemplos para COVID-19 y probamos con la mejor configuración obtenida:
print("********************************* Prueba clasificador con más ejemplos en COVID-19 ********************************* ")
copiaImagenes(pathImagenesExtra,pathCovid)
clasificador = LogisticRegression(solver='liblinear',penalty="l1",C=100)
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[60], norm=True)
genera_confusion(Xtrain,y_train)
genera_confusion(Xval,y_val)
genera_confusion(Xtest,y_test)

¡Error! El path especificado no ha sido encontrado
¡Error! El path especificado no ha sido encontrado
¡Error! El path especificado no ha sido encontrado


TypeError: len() of unsized object

# Segunda aproximación del problema

In [14]:
def obtener_caracteristicas_hog(X):
    n = X.shape[0]
    
    hog_descriptor = np.empty((n,2048))
    for i in range(0,n):
        fd = hog(X[i], orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualize=False, multichannel=False)
        hog_descriptor[i] = fd.reshape(1,-1)
    return hog_descriptor

In [15]:
#Primera prueba de clasificación utilizando todas las características disponibles:
pathCovid = "Data/COVID-19/"
pathNormal = "Data/NORMAL/"
pathViral = "Data/Viral Pneumonia/" 
pathImagenes= "Dataset/"
fichero_train= "train.txt"
fichero_test= "test.txt"
fichero_val= "val.txt"
dx=256 
dy=256

train, y_train = lectura("train/",fichero_train,pathImagenes)
val, y_val = lectura("val/",fichero_val,pathImagenes)
test, y_test = lectura("test/",fichero_test,pathImagenes)

X_train = obtener_caracteristicas_hog(train)
X_val = obtener_caracteristicas_hog(val)
X_test = obtener_caracteristicas_hog(test)

In [17]:
print(X_train.shape)

(5228, 2048)
